In [ ]:
import configparser
import os

def get_onenote_link_from_url_file(url_file_path):
    config = configparser.ConfigParser()
    # On some systems, configparser expects section headers with no trailing spaces,
    # so let's handle potential encoding/format issues carefully:
    with open(url_file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    # Convert line endings or fix formatting if needed
    lines = content.splitlines()
    # Now read the content from memory
    config.read_string("\n".join(lines))

    # The link is usually under the [InternetShortcut] section with the key 'URL'
    return config['InternetShortcut']['URL']

# Example usage:
if __name__ == "__main__":
    url_file_path = "/Users/davidshevchenko/Library/CloudStorage/OneDrive-Personal/Documents/Christianity.url"
    onenote_link = get_onenote_link_from_url_file(url_file_path)
    print("Extracted OneNote link:", onenote_link)


In [ ]:
import msal
import requests

##############################################################################
# Step 1: Configure credentials and create a function to acquire an access token
##############################################################################


CLIENT_ID = "YOUR_CLIENT_ID"
TENANT_ID = "YOUR_TENANT_ID"
CLIENT_SECRET = "YOUR_Client_ID"
AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"

SCOPES = ["Notes.Read", "Notes.ReadWrite"]

def get_access_token():
    """Acquires an access token via device code flow using the MSAL PublicClientApplication."""
    app = msal.PublicClientApplication(
        client_id=CLIENT_ID,
        authority=AUTHORITY
    )

    # 1) First, try to get a token silently from the MSAL cache (if you have previously authenticated)
    accounts = app.get_accounts()
    if accounts:
        result = app.acquire_token_silent(SCOPES, account=accounts[0])
        if result and "access_token" in result:
            return result["access_token"]

    # 2) If no cached token or silent request fails, use device code flow
    flow = app.initiate_device_flow(scopes=SCOPES)
    if "user_code" not in flow:
        raise Exception("Device flow failed to start. Check client ID and scopes.")

    print(f"To sign in, use a web browser to navigate to:\n  {flow['verification_uri']}")
    print(f"Then enter the code:\n  {flow['user_code']}\n")

    result = app.acquire_token_by_device_flow(flow)

    if "access_token" in result:
        return result["access_token"]
    else:
        raise Exception(f"Failed to acquire token: {result.get('error_description')}")

##############################################################################
# Step 2: Helper functions to call Microsoft Graph OneNote endpoints
##############################################################################

def list_notebooks(access_token):
    """Lists all OneNote notebooks in the signed-in user's account."""
    url = "https://graph.microsoft.com/v1.0/me/onenote/notebooks"
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.get(url, headers=headers)
    response.raise_for_status()

    data = response.json()
    notebooks = data.get("value", [])

    print("=== OneNote Notebooks ===")
    for nb in notebooks:
        print(f"- Name: {nb.get('displayName')} | ID: {nb.get('id')}")
    print()
    return notebooks

def list_sections(access_token, notebook_id):
    """Lists the sections within a specific notebook."""
    url = f"https://graph.microsoft.com/v1.0/me/onenote/notebooks/{notebook_id}/sections"
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.get(url, headers=headers)
    response.raise_for_status()

    data = response.json()
    sections = data.get("value", [])

    print("=== Sections in Notebook ===")
    for sec in sections:
        print(f"- Section: {sec.get('displayName')} | ID: {sec.get('id')}")
    print()
    return sections

def list_pages(access_token, section_id):
    """Lists the pages within a specific section."""
    url = f"https://graph.microsoft.com/v1.0/me/onenote/sections/{section_id}/pages"
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.get(url, headers=headers)
    response.raise_for_status()

    data = response.json()
    pages = data.get("value", [])

    print("=== Pages in Section ===")
    for pg in pages:
        print(f"- Page: {pg.get('title')} | ID: {pg.get('id')}")
    print()
    return pages

##############################################################################
# Step 3: Putting it all together
##############################################################################

if __name__ == "__main__":
    # 1) Acquire an access token interactively via device code flow
    token = get_access_token()

    # 2) List all notebooks
    notebooks_list = list_notebooks(token)

    if notebooks_list:
        notebook_id = notebooks_list[0].get("id")
        # 4) List sections in that notebook
        sections_list = list_sections(token, notebook_id)

        # 5) Pick a section ID, list pages
        if sections_list:
            section_id = sections_list[0].get("id")
            list_pages(token, section_id)
        else:
            print("No sections found in the selected notebook.")
    else:
        print("No notebooks found.")
